In [37]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd

In [39]:
slp_score_data = pd.read_csv("../EDA/slp_score_data.csv")
sleep_data = pd.read_csv("../EDA/sleep_data.csv")
wellness_data = pd.read_csv("../EDA/wellness_data.csv")

processed_summary= pd.read_csv("../EDA/sleep_processed_summary.csv")
processed_data= pd.read_csv("../EDA/sleep_processed_data.csv")
processed_shortdata= pd.read_csv("../EDA/sleep_processed_shortdata.csv")

sleepDay = pd.read_csv("../LLM-testing/sleepDay_merged.csv")
dailyActivity = pd.read_csv("../LLM-testing/dailyActivity_merged.csv")

In [41]:
def synthesiser(data, id_col, participant_col, start_participant_num, num_new_participants):
    """
    Generate synthetic data for new participants using CTGAN.

    Parameters:
    - data (pd.DataFrame): Original dataset.
    - id_col (str): Column name to be treated as unique row ID (e.g., 'row_id').
    - participant_col (str): Column name representing participant IDs.
    - start_participant_num (int): Number to start naming new participants from (e.g., 17 for 'p17').
    - num_new_participants (int): How many new participants to synthesize.

    Returns:
    - pd.DataFrame: Synthetic data for the new participants.
    """
    
    # Step 0: Assign unique row ID if not already
    if id_col not in data.columns:
        data[id_col] = range(len(data))
    
    # Step 1: Setup metadata
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)
    metadata.update_column(column_name=id_col, sdtype='id')
    metadata.set_primary_key(column_name=id_col)
    metadata.update_column(column_name=participant_col, sdtype='categorical')

    # Step 2: Fit the synthesizer
    synthesizer = CTGANSynthesizer(metadata)
    synthesizer.fit(data)

    # Step 3: Estimate rows per participant and sample synthetic data
    original_participant_count = data[participant_col].nunique()
    rows_per_participant = len(data) // original_participant_count
    synthetic_sample_size = rows_per_participant * num_new_participants
    synthetic_data = synthesizer.sample(synthetic_sample_size)

    # Step 4: Generate new participant IDs
    new_ids = [f"{i}" for i in range(start_participant_num, start_participant_num + num_new_participants)]

    # Step 5: Build new synthetic dataset
    new_participant_data = pd.DataFrame()

    for new_id in new_ids:
        subset = synthetic_data.sample(rows_per_participant, replace=True).copy()
        subset[participant_col] = new_id
        new_participant_data = pd.concat([new_participant_data, subset], ignore_index=True)

    # Step 6: Reassign row IDs
    new_participant_data[id_col] = range(1, len(new_participant_data) + 1)

    return new_participant_data

In [43]:
sleepDay.describe()

,Id,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
count,4.130000e+02,413.000000,413.000000,413.000000
mean,5.000979e+09,1.118644,419.467312,458.639225
std,2.060360e+09,0.345521,118.344679,127.101607
min,1.503960e+09,1.000000,58.000000,61.000000
25%,3.977334e+09,1.000000,361.000000,403.000000
50%,4.702922e+09,1.000000,433.000000,463.000000
75%,6.962181e+09,1.000000,490.000000,526.000000
max,8.792010e+09,3.000000,796.000000,961.000000


In [45]:
synthetic_sleepDay = synthesiser(
    data=sleepDay,
    id_col='row_id',
    participant_col='Id',
    start_participant_num=34,
    num_new_participants=66
)
sleepDay_final = pd.concat([sleepDay, synthetic_sleepDay], ignore_index=True)
sleepDay_final.describe()
sleepDay_final.to_csv("GAN_generated_data/KaggleDataset/sleepDay_synthesised.csv", index=False)

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [47]:
dailyActivity.describe()

,Id,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
count,9.400000e+02,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000
mean,4.855407e+09,7637.910638,5.489702,5.475351,0.108171,1.502681,0.567543,3.340819,0.001606,21.164894,13.564894,192.812766,991.210638,2303.609574
std,2.424805e+09,5087.150742,3.924606,3.907276,0.619897,2.658941,0.883580,2.040655,0.007346,32.844803,19.987404,109.174700,301.267437,718.166862
min,1.503960e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.320127e+09,3789.750000,2.620000,2.620000,0.000000,0.000000,0.000000,1.945000,0.000000,0.000000,0.000000,127.000000,729.750000,1828.500000
50%,4.445115e+09,7405.500000,5.245000,5.245000,0.000000,0.210000,0.240000,3.365000,0.000000,4.000000,6.000000,199.000000,1057.500000,2134.000000
75%,6.962181e+09,10727.000000,7.712500,7.710000,0.000000,2.052500,0.800000,4.782500,0.000000,32.000000,19.000000,264.000000,1229.500000,2793.250000
max,8.877689e+09,36019.000000,28.030001,28.030001,4.942142,21.920000,6.480000,10.710000,0.110000,210.000000,143.000000,518.000000,1440.000000,4900.000000


In [49]:
synthetic_dailyActivity = synthesiser(
    data=dailyActivity,
    id_col='row_id',
    participant_col='Id',
    start_participant_num=34,
    num_new_participants=66
)
dailyActivity_final = pd.concat([dailyActivity, synthetic_dailyActivity], ignore_index=True)
dailyActivity_final.describe()
dailyActivity_final.to_csv("GAN_generated_data/KaggleDataset/dailyActivity_synthesised.csv", index=False)

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [5]:
synthetic_wellness = synthesiser(
    data=wellness_data,
    id_col='row_id',
    participant_col='participant_id',
    start_participant_num=17,
    num_new_participants=84
)
wellness_final = pd.concat([wellness_data, synthetic_wellness], ignore_index=True)
wellness_final.describe()

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'effective_time_frame'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,fatigue,mood,readiness,sleep_duration_h,sleep_quality,soreness,stress,row_id
count,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000
mean,2.555810,3.110337,4.675044,6.812804,3.043658,2.723746,2.649454,3984.763551
std,0.970288,1.009325,2.260581,1.332907,0.984992,0.888269,1.012589,2784.902112
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,3.000000,6.000000,2.000000,2.000000,2.000000,1363.000000
50%,3.000000,3.000000,5.000000,7.000000,3.000000,3.000000,3.000000,3705.000000
75%,3.000000,4.000000,6.000000,7.000000,4.000000,3.000000,3.000000,6430.500000
max,5.000000,5.000000,10.000000,12.000000,5.000000,5.000000,5.000000,9156.000000


In [6]:
wellness_final.to_csv("GAN_generated_data/wellness_synthesised_100_participants.csv", index=False)

In [7]:
synthetic_slp_score = synthesiser(
    data=slp_score_data,
    id_col='sleep_log_entry_id',
    participant_col='participant_id',
    start_participant_num=17,
    num_new_participants=84
)
slp_score_final = pd.concat([slp_score_data, synthetic_slp_score], ignore_index=True)
slp_score_final.describe()

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'sleep_log_entry_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
count,1.141200e+04,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000
mean,4.098343e+09,71.449264,19.561251,18.990887,37.514108,66.824658,57.343761,0.086773
std,9.363037e+09,9.381680,2.403080,3.271264,5.675605,31.710996,7.818489,0.043431
min,1.000000e+00,35.000000,12.000000,5.000000,3.000000,0.000000,44.000000,0.015385
25%,2.853750e+03,66.000000,17.000000,18.000000,34.000000,45.000000,51.000000,0.055720
50%,5.706500e+03,72.000000,20.000000,19.000000,39.000000,66.000000,59.000000,0.079653
75%,8.559250e+03,78.000000,21.000000,21.000000,41.000000,87.000000,63.000000,0.109593
max,2.654316e+10,94.000000,25.000000,25.000000,47.000000,183.000000,76.000000,0.294766


In [8]:
slp_score_final.to_csv("GAN_generated_data/sleep_score_synthesised_100_participants.csv", index=False)

In [9]:
sleep_data = sleep_data.drop(columns=["levels"])

synthetic_overall_sleep = synthesiser(
    data=sleep_data,
    id_col='row_id',
    participant_col='participant_id',
    start_participant_num=17,
    num_new_participants=84
)
overall_sleep_final = pd.concat([sleep_data, synthetic_overall_sleep], ignore_index=True)
overall_sleep_final.describe()

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,logId,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,infoCode,row_id
count,1.290000e+04,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.00000
mean,2.556849e+10,340.549535,0.864961,343.828295,38.307519,0.504031,350.353488,95.150465,0.459380,4716.58000
std,5.925099e+08,165.347198,4.289623,156.114124,28.195574,1.505830,165.689290,3.926894,0.817274,3295.99718
min,2.447235e+10,60.000000,0.000000,39.000000,0.000000,0.000000,60.000000,0.000000,0.000000,0.00000
25%,2.508231e+10,219.000000,0.000000,265.750000,15.000000,0.000000,253.000000,93.000000,0.000000,1612.75000
50%,2.554535e+10,360.000000,0.000000,384.000000,38.000000,0.000000,385.000000,96.000000,0.000000,4386.50000
75%,2.608390e+10,457.000000,0.000000,453.000000,57.000000,0.000000,464.000000,98.000000,0.000000,7611.25000
max,2.655419e+10,953.000000,35.000000,823.000000,170.000000,33.000000,953.000000,100.000000,2.000000,10836.00000


In [10]:
overall_sleep_final.to_csv("GAN_generated_data/overall_sleep_synthesised_100_participants.csv", index=False)

# initial evaluations

In [25]:
slp_score_data = pd.read_csv("../EDA/slp_score_data.csv")
sleep_data = pd.read_csv("../EDA/sleep_data.csv")
wellness_data = pd.read_csv("../EDA/wellness_data.csv")

wellness_synthesised =  pd.read_csv("GAN_generated_data/wellness_synthesised_100_participants.csv")
slp_score_synthesised =  pd.read_csv("GAN_generated_data/sleep_score_synthesised_100_participants.csv")
sleep_synthesised =  pd.read_csv("GAN_generated_data/overall_sleep_synthesised_100_participants.csv")

In [49]:
shapes_df = pd.DataFrame({
    "Dataset": [
        "slp_score_data",
        "sleep_data",
        "wellness_data",
        "slp_score_synthesised",
        "sleep_synthesised",
        "wellness_synthesised"
    ],
    "Shape": [
        slp_score_data.shape,
        sleep_data.shape,
        wellness_data.shape,
        slp_score_synthesised.shape,
        sleep_synthesised.shape,
        wellness_synthesised.shape
    ]
})

print(shapes_df)

                 Dataset        Shape
0         slp_score_data   (1836, 11)
1             sleep_data   (2064, 17)
2          wellness_data   (1747, 11)
3  slp_score_synthesised  (11412, 11)
4      sleep_synthesised  (12900, 17)
5   wellness_synthesised  (10903, 12)


In [27]:
slp_score_data.describe()

,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
count,1.836000e+03,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000
mean,2.547399e+10,76.460784,19.247277,18.960784,38.252723,73.258170,58.583878,0.090687
std,5.802740e+08,7.795554,2.390916,3.316146,5.146081,27.728065,7.089620,0.037511
min,2.447239e+10,35.000000,12.000000,5.000000,3.000000,0.000000,44.000000,0.015385
25%,2.496905e+10,72.000000,17.000000,17.000000,35.000000,55.000000,53.000000,0.062388
50%,2.543212e+10,77.000000,19.500000,20.000000,39.000000,72.500000,59.000000,0.084501
75%,2.596353e+10,82.000000,21.000000,21.000000,42.000000,91.000000,65.000000,0.110432
max,2.654316e+10,94.000000,25.000000,25.000000,47.000000,183.000000,76.000000,0.294766


In [47]:
slp_score_synthesised.describe()

,sleep_log_entry_id,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
count,1.141200e+04,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000,11412.000000
mean,4.098343e+09,71.449264,19.561251,18.990887,37.514108,66.824658,57.343761,0.086773
std,9.363037e+09,9.381680,2.403080,3.271264,5.675605,31.710996,7.818489,0.043431
min,1.000000e+00,35.000000,12.000000,5.000000,3.000000,0.000000,44.000000,0.015385
25%,2.853750e+03,66.000000,17.000000,18.000000,34.000000,45.000000,51.000000,0.055720
50%,5.706500e+03,72.000000,20.000000,19.000000,39.000000,66.000000,59.000000,0.079653
75%,8.559250e+03,78.000000,21.000000,21.000000,41.000000,87.000000,63.000000,0.109593
max,2.654316e+10,94.000000,25.000000,25.000000,47.000000,183.000000,76.000000,0.294766


In [53]:
ranges = {
    "p01–p16": range(1, 17),
    "p01–p20": range(1, 21),
    "p01–p40": range(1, 41),
    "p01–p60": range(1, 61),
    "p01–p80": range(1, 81),
    "p01–p100": range(1, 101),
}

# Compare describe() for the 'slp_score' column across ranges
summary = pd.DataFrame({
    label: slp_score_synthesised[
        slp_score_synthesised['participant_id'].isin([f'p{str(i).zfill(2)}' for i in rng])
    ]['overall_score'].describe()
    for label, rng in ranges.items()
})

summary

,p01–p16,p01–p20,p01–p40,p01–p60,p01–p80,p01–p100
count,1836.000000,2292.000000,4572.000000,6852.000000,9132.000000,11412.000000
mean,76.460784,75.261344,73.027122,72.147110,71.698533,71.449264
std,7.795554,8.450602,9.119334,9.311064,9.346569,9.381680
min,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
25%,72.000000,70.000000,68.000000,67.000000,66.000000,66.000000
50%,77.000000,76.000000,74.000000,73.000000,72.000000,72.000000
75%,82.000000,81.000000,79.000000,79.000000,78.000000,78.000000
max,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000


In [31]:
sleep_data.describe()

,logId,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,infoCode
count,2.064000e+03,2064.000000,2064.000000,2064.000000,2064.000000,2064.000000,2064.000000,2064.000000,2064.000000
mean,2.547562e+10,444.038275,0.036337,387.182171,56.526647,0.744671,444.038275,94.335271,0.140504
std,5.864598e+08,131.210291,0.889502,111.897713,25.232016,2.085014,131.210291,4.588759,0.506503
min,2.447235e+10,60.000000,0.000000,39.000000,0.000000,0.000000,60.000000,0.000000,0.000000
25%,2.496629e+10,396.000000,0.000000,348.000000,42.000000,0.000000,396.000000,92.000000,0.000000
50%,2.543027e+10,461.500000,0.000000,401.000000,55.000000,0.000000,461.500000,95.000000,0.000000
75%,2.596442e+10,518.000000,0.000000,453.000000,70.000000,0.000000,518.000000,97.000000,0.000000
max,2.655419e+10,953.000000,35.000000,823.000000,170.000000,33.000000,953.000000,100.000000,2.000000


In [33]:
sleep_synthesised.describe()

,logId,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,infoCode,row_id
count,1.290000e+04,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.000000,12900.00000
mean,2.556849e+10,340.549535,0.864961,343.828295,38.307519,0.504031,350.353488,95.150465,0.459380,4716.58000
std,5.925099e+08,165.347198,4.289623,156.114124,28.195574,1.505830,165.689290,3.926894,0.817274,3295.99718
min,2.447235e+10,60.000000,0.000000,39.000000,0.000000,0.000000,60.000000,0.000000,0.000000,0.00000
25%,2.508231e+10,219.000000,0.000000,265.750000,15.000000,0.000000,253.000000,93.000000,0.000000,1612.75000
50%,2.554535e+10,360.000000,0.000000,384.000000,38.000000,0.000000,385.000000,96.000000,0.000000,4386.50000
75%,2.608390e+10,457.000000,0.000000,453.000000,57.000000,0.000000,464.000000,98.000000,0.000000,7611.25000
max,2.655419e+10,953.000000,35.000000,823.000000,170.000000,33.000000,953.000000,100.000000,2.000000,10836.00000


In [55]:
ranges = {
    "p01–p16": range(1, 17),
    "p01–p20": range(1, 21),
    "p01–p40": range(1, 41),
    "p01–p60": range(1, 61),
    "p01–p80": range(1, 81),
    "p01–p100": range(1, 101),
}

# Compare describe() for the 'slp_score' column across ranges
summary = pd.DataFrame({
    label: sleep_synthesised[
        sleep_synthesised['participant_id'].isin([f'p{str(i).zfill(2)}' for i in rng])
    ]['duration'].describe()
    for label, rng in ranges.items()
})

summary

,p01–p16,p01–p20,p01–p40,p01–p60,p01–p80,p01–p100
count,2064.000000,2580.000000,5160.000000,7740.000000,10320.000000,12900.000000
mean,444.038275,418.395736,370.385271,353.522739,345.489244,340.549535
std,131.210291,146.012053,160.523435,164.308385,165.407477,165.347198
min,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
25%,396.000000,361.000000,283.750000,249.000000,228.750000,219.000000
50%,461.500000,443.000000,396.000000,377.000000,366.000000,360.000000
75%,518.000000,509.000000,482.000000,470.000000,462.000000,457.000000
max,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000


In [35]:
wellness_data.describe()

,fatigue,mood,readiness,sleep_duration_h,sleep_quality,soreness,stress
count,1747.000000,1747.000000,1747.000000,1747.000000,1747.000000,1747.000000,1747.000000
mean,2.709216,3.195764,4.946766,6.717802,3.013738,2.826560,2.906125
std,0.679550,0.640844,1.926788,1.338879,0.752114,0.601853,0.722968
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,4.000000,6.000000,3.000000,3.000000,3.000000
50%,3.000000,3.000000,5.000000,7.000000,3.000000,3.000000,3.000000
75%,3.000000,4.000000,6.000000,8.000000,3.000000,3.000000,3.000000
max,5.000000,5.000000,10.000000,12.000000,5.000000,5.000000,5.000000


In [37]:
wellness_synthesised.describe()

,fatigue,mood,readiness,sleep_duration_h,sleep_quality,soreness,stress,row_id
count,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000,10903.000000
mean,2.555810,3.110337,4.675044,6.812804,3.043658,2.723746,2.649454,3984.763551
std,0.970288,1.009325,2.260581,1.332907,0.984992,0.888269,1.012589,2784.902112
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,3.000000,6.000000,2.000000,2.000000,2.000000,1363.000000
50%,3.000000,3.000000,5.000000,7.000000,3.000000,3.000000,3.000000,3705.000000
75%,3.000000,4.000000,6.000000,7.000000,4.000000,3.000000,3.000000,6430.500000
max,5.000000,5.000000,10.000000,12.000000,5.000000,5.000000,5.000000,9156.000000


In [57]:
ranges = {
    "p01–p16": range(1, 17),
    "p01–p20": range(1, 21),
    "p01–p40": range(1, 41),
    "p01–p60": range(1, 61),
    "p01–p80": range(1, 81),
    "p01–p100": range(1, 101),
}

# Compare describe() for the 'slp_score' column across ranges
summary = pd.DataFrame({
    label: wellness_synthesised[
        wellness_synthesised['participant_id'].isin([f'p{str(i).zfill(2)}' for i in rng])
    ]['sleep_quality'].describe()
    for label, rng in ranges.items()
})

summary

,p01–p16,p01–p20,p01–p40,p01–p60,p01–p80,p01–p100
count,1747.000000,2183.000000,4363.000000,6543.000000,8723.000000,10903.000000
mean,3.013738,3.022904,3.046528,3.049824,3.049639,3.043658
std,0.752114,0.809786,0.915198,0.953662,0.977123,0.984992
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,2.000000,2.000000,2.000000,2.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
